## Importing libs

In [1]:
import re
from geopy import geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import glob, os, time
import sys  
cwd = os.getcwd()
from Script_utils import *

## Get all songs from U.S. and list separate articles

In [2]:
base_URL = 'https://en.wikipedia.org'
all_songs_URL = base_URL + '/wiki/List_of_songs_about_cities'

In [3]:
page_soup = load_page(all_songs_URL)

us_h2 = page_soup.select('h2 > span.mw-headline#United_States')[0].parent

In [4]:
cities = []
states = []
songs = []


for element in us_h2.next_siblings:

    if element.name == 'h2': break;
    if element.name == 'h3':
        city_state = get_city_and_state(element.getText())
        start_point = element

        for list_element in start_point.next_siblings:

            if list_element.name == 'h3': break
            if list_element.name == 'ul':

                for song_element in list_element.select('li'):
                    song_name = get_song_name(song_element.getText())
                    songs.append(song_name)
                    cities.append(city_state[0])
                    states.append(city_state[1])

            if list_element.name == 'div':
                single_aricle_URL = list_element.select('a',herf=True)[0]['href']
                print(single_aricle_URL)
                continue

/wiki/List_of_songs_about_Atlanta
/wiki/List_of_songs_about_Birmingham,_Alabama
/wiki/List_of_songs_about_Boston
/wiki/List_of_songs_about_Chicago
/wiki/List_of_songs_about_Detroit
/wiki/List_of_songs_about_Los_Angeles
/wiki/List_of_songs_about_Miami
/wiki/List_of_songs_about_Nashville
/wiki/List_of_songs_about_New_Orleans
/wiki/List_of_songs_about_New_York_City
/wiki/List_of_songs_about_Portland,_Oregon
/wiki/List_of_songs_about_the_San_Francisco_Bay_Area
/wiki/List_of_songs_about_Seattle


- /wiki/List_of_songs_about_Atlanta ✅
- /wiki/List_of_songs_about_Birmingham,_Alabama 🚧
- /wiki/List_of_songs_about_Boston ✅
- /wiki/List_of_songs_about_Chicago ✅
- /wiki/List_of_songs_about_Detroit ✅
- /wiki/List_of_songs_about_Los_Angeles 🚧
- /wiki/List_of_songs_about_Miami ✅
- /wiki/List_of_songs_about_Nashville ✅
- /wiki/List_of_songs_about_New_Orleans ✅
- /wiki/List_of_songs_about_New_York_City ✅
- /wiki/List_of_songs_about_Portland,_Oregon ✅
- /wiki/List_of_songs_about_Seattle 🚧

✅ - csv is generated <br/>
🚧 - work in progress

In [5]:
print(f'Songs {len(songs)} | Cities: {len(cities)} | States: {len(states)}')

Songs 666 | Cities: 666 | States: 666


In [6]:
export_data = pd.DataFrame(data={'City':cities,'State':states,'Song':songs})

In [7]:
export_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    666 non-null    object
 1   State   666 non-null    object
 2   Song    666 non-null    object
dtypes: object(3)
memory usage: 15.7+ KB


In [8]:
rows_with_nan = export_data[export_data.isnull().any(axis=1)]
print(rows_with_nan)
print(f'# of rows with NaN: {len(rows_with_nan)}')

Empty DataFrame
Columns: [City, State, Song]
Index: []
# of rows with NaN: 0


There were 18 rows with NaN values coused by issues below:

1. Missing cloasing quote
- Details: I noticed some of the songs in the Wikipedia article don't have closing quote (!) and my regex fail.
- Solution: I could make more complex regex to handle this situation, but instead I edit Wikipedia article. There are no so many of that rows and it'll help future scrapers :)

2. Different quote type
- Details: I found out there are many different type of quotation marks in UNICODE.
- Solution: I need to sanitize song names a bit

In [9]:
export_data.to_csv(cwd + '/datasets/Data_main.csv',index=False)
export_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    666 non-null    object
 1   State   666 non-null    object
 2   Song    666 non-null    object
dtypes: object(3)
memory usage: 15.7+ KB


## Scraping data for each big city

Unfortunately, every article has a slightly different structure so I had to scrape them separately.

In [10]:
def scrape_single_aricle(URL, start_tag,start_id, stop_tag, city_name, state_name):
    page_soup = load_page(base_URL + URL)

    start = page_soup.find(start_tag,id=start_id).parent

    songs = []
    cities = []
    states = []

    for element in start.next_siblings:
        if element.name == stop_tag: break
        if element.name == 'ul':
            for li in element.select('li'):
                song_name = get_song_name(li.getText())
                songs.append(song_name)
                cities.append(city_name)
                states.append(state_name)
    return {
        'songs':songs,
        'cities':cities,
        'states':states
    }

### Detroit

In [11]:
detroit_results = scrape_single_aricle(
    URL = '/wiki/List_of_songs_about_Detroit',
    start_tag = 'span',
    start_id='0-9',
    stop_tag = 'div',
    city_name = 'Detroit',
    state_name = 'Michigan')

In [12]:
export_data_detroit = pd.DataFrame(data={'City':detroit_results['cities'],'State':detroit_results['states'],'Song':detroit_results['songs']})
export_data_detroit.to_csv(cwd + '/datasets/Data_detroit.csv',index=False)
export_data_detroit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    316 non-null    object
 1   State   316 non-null    object
 2   Song    303 non-null    object
dtypes: object(3)
memory usage: 7.5+ KB


### Miami

In [13]:
miami_results = scrape_single_aricle(
    URL = '/wiki/List_of_songs_about_Miami',
    start_tag = 'span',
    start_id='Songs_about_Miami',
    stop_tag = 'h2',
    city_name = 'Miami',
    state_name = 'Florida')

In [14]:
export_data_miami = pd.DataFrame(data={'City':miami_results['cities'],'State':miami_results['states'],'Song':miami_results['songs']})
export_data_miami.to_csv(cwd + '/datasets/Data_miami.csv',index=False)
export_data_miami.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    45 non-null     object
 1   State   45 non-null     object
 2   Song    45 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


### New Orleans

In [15]:
new_orleans_results = scrape_single_aricle(
    URL = '/wiki/List_of_songs_about_New_Orleans',
    start_tag = 'span',
    start_id='0-9',
    stop_tag = 'div',
    city_name = 'New Orleans',
    state_name = 'Louisiana')

In [16]:
export_data_new_orleans = pd.DataFrame(data={'City':new_orleans_results['cities'],'State':new_orleans_results['states'],'Song':new_orleans_results['songs']})
export_data_new_orleans.to_csv(cwd + '/datasets/Data_new_orleans.csv',index=False)
export_data_new_orleans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    779 non-null    object
 1   State   779 non-null    object
 2   Song    775 non-null    object
dtypes: object(3)
memory usage: 18.4+ KB


### Los Angeles

In [17]:
los_angeles_results = scrape_single_aricle(
    URL = '/wiki/List_of_songs_about_Los_Angeles',
    start_tag = 'span',
    start_id='#s–A',
    stop_tag = 'h2',
    city_name = 'Los Angeles',
    state_name = 'California')

In [18]:
export_data_los_angeles = pd.DataFrame(data={'City':los_angeles_results['cities'],'State':los_angeles_results['states'],'Song':los_angeles_results['songs']})
export_data_los_angeles.to_csv(cwd + '/datasets/Data_los_angeles.csv',index=False)
export_data_los_angeles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    2696 non-null   object
 1   State   2696 non-null   object
 2   Song    2681 non-null   object
dtypes: object(3)
memory usage: 63.3+ KB


### Chicago

In [19]:
chicago_results = scrape_single_aricle(
    URL = '/wiki/List_of_songs_about_Chicago',
    start_tag = 'span',
    start_id='0–9',
    stop_tag = 'p',
    city_name = 'Chicago',
    state_name = 'Illinois')

In [20]:
export_data_chicago = pd.DataFrame(data={'City':chicago_results['cities'],'State':chicago_results['states'],'Song':chicago_results['songs']})
export_data_chicago.to_csv(cwd + '/datasets/Data_chicago.csv',index=False)
export_data_chicago.info()

### New York City

In [21]:
page_soup = load_page(base_URL + '/wiki/List_of_songs_about_New_York_City')

start = page_soup.find("span",id="0–9").parent

songs = []
cities = []
states = []

for element in start.next_siblings:
    if element.name == 'p': break
    if element.name == 'div':
        el = element.select('ul')[0]
        for li in el.select('li'):
            song_name = get_song_name(li.getText())
            songs.append(song_name)
            cities.append('New York City')
            states.append('New York')

In [22]:
export_data_new_york_city = pd.DataFrame(data={'City':cities,'State':states,'Song':songs})
export_data_new_york_city.to_csv(cwd + '/datasets/Data_new_york_city.csv',index=False)
export_data_new_york_city.info()

### Nashville

In [23]:
page_soup = load_page(base_URL + '/wiki/List_of_songs_about_Nashville,_Tennessee')

start = page_soup.find("span",id="C").parent

songs = []
cities = []
states = []

for element in start.next_siblings:
    if element.name == 'h2' and element.select('span#References'): break
    if element.name == 'ul':
        for li in element.select('li'):
            song_name = get_song_name(li.getText())
            songs.append(song_name)
            cities.append('Nashville')
            states.append('Tennessee')

In [24]:
export_data_nashville = pd.DataFrame(data={'City':cities,'State':states,'Song':songs})
export_data_nashville.to_csv(cwd + '/datasets/Data_nashville.csv',index=False)
export_data_nashville.info()

### Atlanta

In [25]:
page_soup = load_page(base_URL + '/wiki/List_of_songs_about_Atlanta')

songs_list = page_soup.find("ul")

songs = []
cities = []
states = []

for li in songs_list.select('li'):
    song_name = get_song_name(li.getText())
    songs.append(song_name)
    cities.append('Atlanta')
    states.append('Georgia')

In [26]:
export_data_atlanta = pd.DataFrame(data={'City':cities,'State':states,'Song':songs})
export_data_atlanta.to_csv(cwd + '/datasets/Data_atlanta.csv',index=False)
export_data_atlanta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    41 non-null     object
 1   State   41 non-null     object
 2   Song    41 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


### Boston

In [27]:
page_soup = load_page(base_URL + '/wiki/List_of_songs_about_Boston')

songs_list = page_soup.find("ul")

songs = []
cities = []
states = []

for li in songs_list.select('li'):
    song_name = get_song_name(li.getText())
    songs.append(song_name)
    cities.append('Boston')
    states.append('Massachusetts')

In [28]:
export_data_boston = pd.DataFrame(data={'City':cities,'State':states,'Song':songs})
export_data_boston.to_csv(cwd + '/datasets/Data_boston.csv',index=False)
export_data_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    134 non-null    object
 1   State   134 non-null    object
 2   Song    134 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


### Portland

In [29]:
page_soup = load_page(base_URL + '/wiki/List_of_songs_about_Portland,_Oregon')

songs_list = page_soup.find("ul")

songs = []
cities = []
states = []

for li in songs_list.select('li'):
    song_name = get_song_name(li.getText())
    songs.append(song_name)
    cities.append('Portland')
    states.append('Oregon')

In [30]:
export_data_portland = pd.DataFrame(data={'City':cities,'State':states,'Song':songs})
export_data_portland.to_csv(cwd + '/datasets/Data_portland.csv',index=False)
export_data_portland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    31 non-null     object
 1   State   31 non-null     object
 2   Song    31 non-null     object
dtypes: object(3)
memory usage: 872.0+ bytes


## Getting lat and long from GeoPy

In [31]:
geolocator = Nominatim(user_agent="my_app")

In [32]:
separate_cities = [
    {
        'city':'Los Angeles',
        'state':'California'
    },
    {
        'city':'Atlanta',
        'state':'Georgia'
    },
    {
        'city':'Boston',
        'state':'Massachusetts'
    },
    {
        'city':'Chicago',
        'state':'Illinois'
    },
    {
        'city':'Detroit',
        'state':'Michigan'
    },
    {
        'city':'Miami',
        'state':'Florida'
    },
    {
        'city':'Nashville',
        'state':'Tennessee'
    },
    {
        'city':'New Orleans',
        'state':'Louisiana'
    },
    {
        'city':'New York City',
        'state':'New York'
    },
    {
        'city':'Portland',
        'state':'Oregon'
    }
]

In [33]:
i = 0
while i < len(separate_cities):
    try:
        loc = geolocator.geocode(f'{separate_cities[i]["city"]},{separate_cities[i]["state"]} United States')
    except Exception:
        print(f'🛑 Can\'t generate coordinates for {separate_cities[i]["city"]}')
        continue

    dataset_to_change_PATH = cwd + f'/datasets/Data_{normalized_city_name(separate_cities[i]["city"])}.csv'

    data_to_change = pd.read_csv(dataset_to_change_PATH, index_col=[0])
    data_to_change['lat'] = loc.latitude
    data_to_change['long'] = loc.longitude
    data_to_change.to_csv(dataset_to_change_PATH)
    print(f'✅ Coordinates for {separate_cities[i]["city"]} generated!')
    i+=1


✅ Coordinates for Los Angeles generated!
✅ Coordinates for Atlanta generated!
✅ Coordinates for Boston generated!
✅ Coordinates for Chicago generated!
✅ Coordinates for Detroit generated!
✅ Coordinates for Miami generated!
✅ Coordinates for Nashville generated!
✅ Coordinates for New Orleans generated!
✅ Coordinates for New York City generated!
✅ Coordinates for Portland generated!



## Get lat and long for main dataset

*Note: GeoPy allows for 2500 requests per day*

In [34]:
main_dataset_import = pd.read_csv(cwd + '/datasets/Data_main.csv')
main_dataset = main_dataset_import.copy()
main_dataset

,City,State,Song
0,Aberdeen,Maryland,I-95
1,Abilene,Kansas,Abilene
2,Abilene,Kansas,Loser
3,Akron,Ohio,Downtown (Akron)
4,Akron,Ohio,My City Was Gone
...,...,...,...
661,Ypsilanti,Michigan,Born in a Trailer
662,Ypsilanti,Michigan,"For The Widows In Paradise, For The Fatherless..."
663,Ypsilanti,Michigan,Ypsilanti
664,Ypsilanti,Michigan,Ypsilanti Song


In [35]:
main_dataset['Location Name'] = main_dataset.apply(lambda row: f'{row["City"]}, {row["State"]} United States', axis=1)
main_dataset

,City,State,Song,Location Name
0,Aberdeen,Maryland,I-95,"Aberdeen, Maryland United States"
1,Abilene,Kansas,Abilene,"Abilene, Kansas United States"
2,Abilene,Kansas,Loser,"Abilene, Kansas United States"
3,Akron,Ohio,Downtown (Akron),"Akron, Ohio United States"
4,Akron,Ohio,My City Was Gone,"Akron, Ohio United States"
...,...,...,...,...
661,Ypsilanti,Michigan,Born in a Trailer,"Ypsilanti, Michigan United States"
662,Ypsilanti,Michigan,"For The Widows In Paradise, For The Fatherless...","Ypsilanti, Michigan United States"
663,Ypsilanti,Michigan,Ypsilanti,"Ypsilanti, Michigan United States"
664,Ypsilanti,Michigan,Ypsilanti Song,"Ypsilanti, Michigan United States"


In [36]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

main_dataset['Geo Location'] = main_dataset['Location Name'].apply(geocode)

main_dataset['lat'] = main_dataset['Geo Location'].apply(lambda loc: loc.latitude if loc else None)
main_dataset['long'] = main_dataset['Geo Location'].apply(lambda loc: loc.longitude if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Baton Rouge, Louisiana United States',), **{}).
Traceback (most recent call last):
  File "/usr/local/Caskroom/miniforge/base/envs/webscraping/lib/python3.8/urllib/request.py", line 1354, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/usr/local/Caskroom/miniforge/base/envs/webscraping/lib/python3.8/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/local/Caskroom/miniforge/base/envs/webscraping/lib/python3.8/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/local/Caskroom/miniforge/base/envs/webscraping/lib/python3.8/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/local/Caskroom/miniforge/base/envs/webscraping/lib/python3.8/http/client.py", line 1007, in _send_output
    self.sen

Removing unused columns for easier merge

In [42]:
main_dataset.info()

,City,State,Song,lat,long
0,Aberdeen,Maryland,I-95,39.509556,-76.164120
1,Abilene,Kansas,Abilene,38.917252,-97.213770
2,Abilene,Kansas,Loser,38.917252,-97.213770
3,Akron,Ohio,Downtown (Akron),41.083064,-81.518485
4,Akron,Ohio,My City Was Gone,41.083064,-81.518485
...,...,...,...,...,...
661,Ypsilanti,Michigan,Born in a Trailer,42.241056,-83.613055
662,Ypsilanti,Michigan,"For The Widows In Paradise, For The Fatherless...",42.241056,-83.613055
663,Ypsilanti,Michigan,Ypsilanti,42.241056,-83.613055
664,Ypsilanti,Michigan,Ypsilanti Song,42.241056,-83.613055


In [41]:

main_dataset.drop(['Location Name','Geo Location'], axis=1, inplace=True)

main_dataset

KeyError: "['Location Name' 'Geo Location'] not found in axis"

In [38]:
main_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    666 non-null    object 
 1   State   666 non-null    object 
 2   Song    666 non-null    object 
 3   lat     665 non-null    float64
 4   long    665 non-null    float64
dtypes: float64(2), object(3)
memory usage: 26.1+ KB


## Merge all datasets

In [39]:
merged_data = pd.concat(map(pd.read_csv, glob.glob('datasets/*.csv')), ignore_index=True)
merged_data.to_csv(cwd + '/Data_merged.csv')
merged_data

,City,State,Song,lat,long
0,Los Angeles,California,10th & Crenshaw,34.053691,-118.242766
1,Los Angeles,California,100 Miles and Runnin',34.053691,-118.242766
2,Los Angeles,California,101 Eastbound,34.053691,-118.242766
3,Los Angeles,California,The 103rd St. Theme,34.053691,-118.242766
4,Los Angeles,California,1977 Sunset Strip,34.053691,-118.242766
...,...,...,...,...,...
9075,Ypsilanti,Michigan,Born in a Trailer,NaN,NaN
9076,Ypsilanti,Michigan,"For The Widows In Paradise, For The Fatherless...",NaN,NaN
9077,Ypsilanti,Michigan,Ypsilanti,NaN,NaN
9078,Ypsilanti,Michigan,Ypsilanti Song,NaN,NaN


In [40]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9080 entries, 0 to 9079
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    9080 non-null   object 
 1   State   9080 non-null   object 
 2   Song    9026 non-null   object 
 3   lat     8414 non-null   float64
 4   long    8414 non-null   float64
dtypes: float64(2), object(3)
memory usage: 354.8+ KB
